# CART 决策树

## 一、简介

CART(Classification & Regression)的简称，分类和回顾任务都可以应用。CART决策树用 `基尼指数` 进行划分选择。

## 二、基尼指数

### 2.1 定义

假设：样本D，样本纯度的基尼指数为：

$$ Gini(D) = \sum_{k=1}^{K} \sum_{k^{'}\neq k}p_{k}p_{k^{'}} = \sum_{k=1}^{K} p_{k}(1 - p_{k}) = \sum_{k=1}^{K}p_{k} - \sum_{k=1}^{K}p_{k}^{2} = 1 - \sum_{k=1}^{K}p_{k}^{2} $$

直观上理解基尼指数，可以认为，随意从数据D中抽取两个样本，起标记不一致的概率。因此，基尼指数越小，则说明数据集纯度越高(样本中可分类的数据越少)。

根据上面的公式，属性 $ a $ 的基尼指数为：

$$ Gini\_index(D, a) = \sum_{v=1}^{V} \frac{|D^{v}|}{|D|}Gini(D^{v}) $$

综上，我们在候选属性集合 $A$ 中，选择那个使得划分后基尼指数最小的属性作为最优划分属性，即 $ a_{*} = \mathop{\arg\min}_{a\in A} Gini\_index(D, a). $

## 三、剪枝处理

剪枝是避免过拟合的主要手段，剪枝的基本策略有'预剪枝'与'后剪枝'。

- 预剪枝：在决策树生成过程中，对每个结点在划分前进行估计，若当前节点的划分不能带来决策树泛化性能提升，则停止划分并将当前节点标记为叶结点。

- 后剪枝：先从训练集中，生成一棵完整的决策树，然后自底向上对非叶节点进行考察，若该节点的子树替换为叶结点能提升泛化性能，则将该子树替换为叶结点。

![西瓜数据](../imgs/西瓜数据集2.png)

取出1/3作为训练集，2/3作为验证集。利用训练集，训练模型。

### 3.1 预剪枝

以ID3中的信息增益指标为例，若在划分之前所有样本集中在根节点。以西瓜数据为例，若不进行划分以根节点作为第一个分类，所有样本被交集为'好瓜'。那么，正确率为3/7 * 100% = 42.9%

![预剪枝](../imgs/预剪枝.png)

在用属性'脐部'划分之后，结点2、3、4标记为 好瓜、好瓜、坏瓜，准确率为 5/7 * 100% = 71.4% > 42.9%，用'脐部'划分得以确定。

然后，一次对2、3、4结点进行递归处理，直至划分后的准确率低于划分前，停止划分。

预剪枝优缺点

- 优点：效率高
- 缺点：容易欠拟合

### 3.2 后剪枝

后剪枝先用训练集生成一棵完整的决策树，如图4.5，决策树准确率为42.9%。

后剪枝先考察图4.5中的，6结点。若将6结点划分，进行剪枝，相当于直接把6替换替换为叶结点，样本编号为{7, 15}，一好一坏，约定标记为好。此时，剪枝后，准确率为57.1%。确定进行剪枝。

然后，考察结点5，剪枝后准确率还是57.1%不变，可以不进行剪枝。

最后得到图4.7。

### 3.3 总结

对比图4.7与图4.6可看出

- 后剪枝决策树通常比预剪枝决策树保留更多的分支。一般情况下，后剪枝决策树的欠拟合风险小，泛化性能往往优于预剪枝决策树。
- 后剪枝的效率比预剪枝要慢。



## 四、缺失与连续值处理

### 4.1 连续值处理

连续属性的可取值不再是有限集合。因此，不能直接根据连续属性的可取值来对结点进行划分。此时，通过连续属性离散化方法来处理。最简单的策略是采用二分法对连续属性进行处理。

给定样本$D$和连续属性$a$，假设$a$在$D$上出现了n个不同的取值，将这些至进行升序排列，记作$ \{ a^{1}, a^{2}, ..., a^{n} \} $ 。基于划分点t可以将D划分为子集 $ D_{t}^{-} $ 和 $ D_{t}^{+} $。其中$ D_{t}^{-} $为在a上取值小于t的样本，$ D_{t}^{+} $为取值大于t的样本。对于相邻的属性取值$a^{i}$与$a^{i+1}$来说，t在区间$[a^{i}, a^{i+1})$中任意取值所产生的划分结果相同。因此，对于连续属性a，我们可考察包含$n - 1$个元素的候选划分点集合，划分为：

$$ T_{a} = \{ \frac{a^{i} + a^{i+1} + 1}{2} | 1 \leqslant i \leqslant n - 1 \} $$

就是以$a^{i}$与$a^{i+1}$的平均值，作为划分点。然后，就可以像离散属性一样来处理连续属性。

以ID3为例，信息增益Gain进行改造可得：

$$ 
\begin{equation}
\begin{aligned}
Gain(D, a) &= \mathop{\max}_{t \in T_{a}} Gain(D, a, t) \\
&= \mathop{\max}_{t \in T_{a}} D - \sum_{\lambda \in \{-, + \}}Ent(D_{t}^{\lambda})
\end{aligned}
\end{equation} 
$$

找出$T_{a}$中，可以让信息增益最大的划分点，用于划分。

注意：
1. 与离散属性不同，若当前结点划分属性为连续属性，该属性还可作为其后代结点的划分属性。例：父结点上应用 密度 <= 0.381，不会禁止在子结点上使用 密度 <= 0.291。

### 4.2 缺失值处理

&nbsp;&nbsp;&nbsp;&nbsp;需要解决的问题：
1. 在属性值缺失的情况下如何进行划分属性选择？
2. 给定划分属性，若样本在该属性上的数据缺失，应该如何进行划分？

#### 4.2.1 问题1

假设：在属性$a$上，对样本$D$做划分。先找到属性$a$上无缺失的样本$\widetilde{D}$。此时，仅可以对$\widetilde{D}$来判断属性$a$的信息增益或基尼系数。

在开始学习阶段，初始化每个样本的权重为1. 若属性$a$有$V$个可取值 $\{a^{1}, ..., a^{v} \}$，样本的权重为$ \omega _{x} $. 

定义：

$$ \rho = \frac{\mathop{\sum}_{x \in \widetilde{D}}\omega_{x}}{\mathop{\sum}_{x \in D}\omega_{x}} $$

$$ \widetilde{p}_{k} = \frac{\mathop{\sum}_{x \in \widetilde{D}_{k}}\omega_{x}}{\mathop{\sum}_{x \in D}\omega_{x}}, k \in [1, |y|] $$

$$ \widetilde{r}_{k} = \frac{\mathop{\sum}_{x \in \widetilde{D}^{v}}\omega_{x}}{\mathop{\sum}_{x \in D}\omega_{x}}, v \in [1, V] $$

&nbsp;&nbsp;&nbsp;&nbsp;其中, 对于属性$a$, $\rho$ 表示 无缺失值样本所占比例，$ \widetilde{p}_{k} $表示无缺失样本中第$k$类所占比例，$ \widetilde{r}_{k} $ 表示无缺失值样本在属性$a$上取值为$a^{v}$的样本所占比例。

信息增益的计算方法推广为：

$$ 
\begin{equation}
\begin{aligned}
Gain(D, a) &= \rho \times Gain(\widetilde{D}, a) \\
&= \rho \times (Ent(\widetilde{D}) - \sum_{v=1}^{V} \widetilde{r}_{k} Ent(\widetilde{D}^{v}))
\end{aligned}
\end{equation} 
$$

其中，

$$ Ent(\widetilde{D}) = -\sum_{k=1}^{K} \widetilde{p}_{k}\log_{2} \widetilde{p}_{k} $$

基尼指数的计算方法推广为:

$$
\begin{equation}
\begin{aligned}
Gini\_index(D, a) &= \rho \times Gini\_index(\widetilde{D}, a) \\
&= \rho \times \sum_{v=1}^{V} \widetilde{r_{v}} Gini(\widetilde{D^{v}})
\end{aligned}
\end{equation} 
$$

其中，

$$ Gini(\widetilde{D}) = 1 - \sum_{k=1}^{K} p^{2}_{k} $$

#### 4.2.2 问题2

&nbsp;&nbsp;&nbsp;&nbsp;若样本在划分属性$a$上，取值已知，那么样本的权重保持为 $ \omega_{x} $；若样本在划分属性$a$上，取值未知，那么将样本x划入所有子节点，且样本权重在属性$a^{v}$对应的子节点上调整为 $\widetilde{r}_{k} \cdot \omega_{x}. $。可以这么理解，就是让在划分属性$a$上，存在缺失值的样本，按不同概率划分到不同的子节点中。

## 五、Demo

In [10]:
import numpy as np
import pandas as pd

from collections import Counter

import sys
sys.path.append("../")

from utils.dataset import load_watermelon_2_alpha

class CART(object):
    def __init__(self, ) -> None:
        # self.shape = (X.shape[0], X.shape[1] + 1)
        self.tree = list()
        super().__init__()
    
    def separate(self, X, y, parent='root'):
        min_gini = 9999
        res_dict = {}

        # if X.shape[0] == 0:
        #     # 若无数据输入
        #     return 1
        # elif y.nunique() == 1:
        #     # 若多类型
        #     return 1

        for _, col in enumerate(X.columns):  # 对每个属性进行划分
            gini_index_a = gini_index(
                X, y, col
            )
            if gini_index_a < min_gini:
                min_gini = gini_index_a
                res_dict['gini'] = gini_index_a
                res_dict['a'] = col
                res_dict['parent'] = parent
                # print(res_dict)
        
        # 继续划分
        a_unique_values = df[res_dict['a']].unique()
        for a_value in a_unique_values:
            v_df = df.loc[df[res_dict['a']]==a_value]
            res_dict['data'] = list(v_df.index)
            res_dict['a_v'] = res_dict['a'] + '-' + a_value
            print(res_dict)

            # update self.tree
            rv = self.separate(
                X=v_df.loc[:, [x for x in list(v_df.columns) if x != 'target']], 
                y=v_df['target'], 
                parent=res_dict['a'] + '-' + a_value
            )
            if rv == 1:
                self.tree.append({'a': res_dict['a'], 'parent': res_dict['parent'], 'counter': dict(Counter(v_df['target']))})
    
    def fit(self, X, y, prune='pre'):
        """
        拟合CART
        """
        self.separate(X, y)


def gini(y) -> float:
    """基尼指数，计算样本纯度

    基尼指数越小，则说明数据集纯度越高(样本中可分类的数据越少)。

    Args:
        y : [样本标签]

    Returns:
        float: [样本纯度对应的基尼指数]]
    """
    target_counter_dict = dict(Counter(y))
    target_size = len(y)

    pk_square_sum = 0
    for target_value in target_counter_dict.keys():
        pk = target_counter_dict[target_value] / target_size
        pk_square_sum += (pk ** 2)
    gini = 1 - pk_square_sum
    return gini

def gini_index(X, y, flag: str) -> float:
    """对属性a的基尼指数

    Args:
        X ([type]): [特征]
        y ([type]): [标签]
        flag (str): [属性a对应的列名]

    Returns:
        float: [属性a的基尼指数]
    """
    d_size = X.shape[0]  # 样本数量
    Av = list(set((X[flag])))  # 在属性a上，a对应的取值列表
    Av_size = len(Av)  # 在属性a上，a对应的取值数量
    Av_counter_dict = dict(Counter(X[flag]))  # 对Av进行统计

    gini_value = 0
    for _, av in enumerate(Av):  # 统计属性a上，每个取值对应的gini指数
        per = Av_counter_dict[av] / d_size
        av_index = np.where(X[flag]==av)[0]  # 样本 在属性a上 取值 为 av 的样本索引

        X_dv = X.loc[av_index, :]
        y_dv = y.loc[av_index]

        # 对缺失值进行处理，测试数据中，av = 0，则为缺失。
        # 将该样本按权重放入所有分支中
        

        dv_gini = gini(y_dv)
        gini_value += (per * dv_gini)
    return gini_value


def pre_prune_fit():
    pass

def after_prune_fit():
    pass


df = load_watermelon_2_alpha()
df.fillna(0, inplace=True)
X = df.loc[:, [x for x in list(df.columns) if x != 'target']]
y = df.target
print(df)

CART().fit(X, y)

    色泽  根蒂  敲声  纹理  脐部  触感  target
0    0  蜷缩  浊响  清晰  凹陷  硬滑       1
1   乌黑  蜷缩  沉闷  清晰  凹陷   0       1
2   乌黑  蜷缩   0  清晰  凹陷  硬滑       1
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑       1
4    0  蜷缩  浊响  清晰  凹陷  硬滑       1
5   青绿  稍蜷  浊响  清晰   0  软粘       1
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘       1
7   乌黑  稍蜷  浊响   0  稍凹  硬滑       1
8   乌黑   0  沉闷  稍糊  稍凹  硬滑       0
9   青绿  硬挺  清脆  清晰  平坦  软粘       0
10  浅白  硬挺  清脆  模糊  平坦   0       0
11  浅白  蜷缩   0  模糊  平坦  软粘       0
12   0  稍蜷  浊响  稍糊  凹陷  硬滑       0
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑       0
14  乌黑  稍蜷  浊响   0   0  软粘       0
15  浅白  蜷缩  浊响  模糊  平坦  硬滑       0
16  青绿   0  沉闷  稍糊  稍凹  硬滑       0
{'gini': 0.253781512605042, 'a': '纹理', 'parent': 'root', 'data': [0, 1, 2, 3, 4, 5, 9], 'a_v': '纹理-清晰'}


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([6], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"